# Notebook 11: Plotting planes on the stereonet

Planes passing through the center of the sphere intersect the sphere along great circles. These are more difficult to draw on the stereonet, but fortunately the rotation routine we covered in notebook 9 provides a simple and elegant way to do it. To draw a great circle, we just need to rotate the strike line of the plane about the pole to the plane, in fixed increments (e.g., 1°) until completing 180°. These incrementally rotated lines describe the path of the great circle.

## Python function

For a plane of a given strike and dip (RHR), the function below returns the coordinates of the great circle on an equal angle or equal area stereonet.

In [1]:
# this makes visible our functions folder
import sys, os
sys.path.append(os.path.abspath("../functions"))

In [2]:
# great_circle function: this function is also in functions/great_circle.py

import numpy as np
from pole import pole_from_plane
from rotate import rotate
from st_coord_line import st_coord_line

def great_circle(strike,dip,stype):
	"""
	great_circle computes the great circle path of a plane
	on an equal angle or equal area stereonet of unit radius
	
	strike = strike of plane
	dip = dip of plane
	stype = Stereonet type: 0 = equal angle, 1 = equal area
	path = x and y coordinates of points in great circle path
	
	NOTE: strike and dip should be entered in radians
			and follow the RHR convention
	
	Python function translated from the Matlab function
	great_circle in Allmendinger et al. (2012)
	"""
	pi = np.pi
	# Compute the pole to the plane. This will be the axis of
	# rotation to make the great circle
	trda, plga = pole_from_plane(strike,dip)
	
	# Now pick the strike line at the intersection of the
	# great circle with the primitive of the stereonet
	trd, plg = strike, 0.0
	
	# To make the great circle, rotate the line 180 degrees
	# in increments of 1 degree
	rot = np.radians(np.arange(0,181,1))
	path = np.zeros((rot.shape[0],2))
	
	for i in range(rot.size):
	# Avoid joining ends of path
		if rot[i] == pi:
			rot[i] = rot[i] * 0.9999
		# Rotate line
		rtrd, rplg = rotate(trda,plga,rot[i],trd,plg,"a")
		# Calculate stereonet coordinates of rotated line
		# and add to great circle path
		path[i,0], path[i,1] = st_coord_line(rtrd,rplg,stype)
	
	return path

## Application:

Now let's plot a plane and the pole to the plane on an equal angle and an equal area stereonet. The code below plots the stereonets side by side and allows setting the strike and dip of the plane interactively using sliders.

In [3]:
def plot_plane(strike, dip):
    """
    Plots a plane and the pole to the plane 
    on an equal angle (left) and equal area (right) stereonet

    Input follows a RHR convention:
    strike = strike of the plane
    dip = dip of the plane

    Output:
    None
    """
    # import necessary libraries
    import matplotlib.pyplot as plt

    # create a figure with two subplots side by side
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # coordinates of primitive circle, radius = 1
    theta = np.radians(np.arange(0, 361, 1))
    x_c = np.cos(theta)
    y_c = np.sin(theta)

    # plot the primitive circle and the line
    for i in range(2):
        # primitive and center
        ax[i].plot(x_c, y_c, 'k')
        ax[i].plot(0, 0, 'k+')
        
        # pole to plane
        trend, plunge = pole_from_plane(np.radians(strike), np.radians(dip))
        xp, yp = st_coord_line(trend, plunge, i)
        ax[i].plot(xp, yp, 'ro')
        
        # plane
        path = great_circle(np.radians(strike), np.radians(dip), i)
        ax[i].plot(path[:,0], path[:,1], 'r-', linewidth=2)

        # axes settings
        ax[i].set_aspect('equal')
        ax[i].set_xlim(-1.1, 1.1)
        ax[i].set_ylim(-1.1, 1.1)
        ax[i].axis('off')
        ax[i].set_title(['Equal angle', 'Equal area'][i] + ' stereonet')
    
    plt.show()

# use ipywidgets to interact with the function
from ipywidgets import interact
interact(plot_plane, strike=(0, 360, 1), dip=(0, 90, 1));

interactive(children=(IntSlider(value=180, description='strike', max=360), IntSlider(value=45, description='di…

This is a fantastic way to illustrate how a plane and its pole plot on a stereonet 😀